In [86]:
a = set((123, 21, 1, 2))
b = set((1, 2))

list(a.intersection(b))

[1, 2]

In [1]:
import glob
import os
import random

try:
    from . import config
except:
    import config

PWD = os.path.dirname(config.__file__)


class GetNewImage:
    ImageFiles = {}
    XmlFiles = {}
    PendingImages = []
    CompletedImages = []

    @staticmethod
    def refresh():
        # with timely invervals - 5 mins update
        GetNewImage.XmlFiles = GetNewImage._get_annotations()
        GetNewImage.ImageFiles = GetNewImage._get_images()

        def trim_ext(x):
            return x.rsplit('.')[-2]
        xml_files_keys = set(map(trim_ext, GetNewImage.XmlFiles.keys()))
        image_files_keys = set(map(trim_ext, GetNewImage.ImageFiles.keys()))
        GetNewImage.PendingImages = list(image_files_keys - xml_files_keys)
        GetNewImage.CompletedImages = list(image_files_keys.intersection(xml_files_keys))

    @staticmethod
    def update_data(data):
        pass

    @staticmethod
    def get_new_image(data=None):
        if data:
            GetNewImage.update_data(data)

        if GetNewImage.PendingImages:
            image_file = random.choice(GetNewImage.PendingImages)
            image_file = [each for each in GetNewImage.ImageFiles.keys() if each.startswith(image_file)][0]
            send_info = {
                'id': image_file,
                'url': GetNewImage.ImageFiles[image_file]['url'],
                'folder': '',
                "annotations": []
            }
        else:
            send_info = {"url": "/static/data/images/pexels-photo-60091.jpg",
                         "id": "pexels-photo-60091.jpg",
                         "folder": "collection_01/part_1",
                         "annotations": [
                             {
                                 "tag": "Eagle",
                                 "x": 475, "y": 225,
                                 "width": 230.555555554,
                                 "height": 438.888888886}
                         ]
                         }
        return send_info

    @staticmethod
    def get_old_image():
        image_file = random.choice(list(GetNewImage.XmlFiles.keys()))
        send_info = {
            'id': image_file,
            'url': GetNewImage.XmlFiles[image_file]['url'],
            'folder': '',
            "annotations": []
        }
        return send_info

    @staticmethod
    def _parser_folder(folder='static/data/images', exts=None):
        search_folder = folder
        if os.path.isdir(os.path.join(PWD, folder)):
            search_folder = os.path.join(PWD, folder)
        # print('Request Parsing: {}'.format(search_folder))
        bag = []
        for filename in glob.iglob(search_folder + '*', recursive=True):
            file = os.path.basename(filename)
            url = os.path.join(folder, file)
            bag.append(url)
        if not exts:
            return bag
        _bag = []
        for ext in exts:
            for file in bag:
                if file.endswith(ext):
                    _bag.append(file)
        return _bag

    @staticmethod
    def _get_images():
        images_dict = {}
        for each in GetNewImage._parser_folder('static/data/images/'):
            url = each.split(PWD)[-1].lstrip('/')
            file = os.path.basename(url)
            # print(url, file)
            images_dict[file] = {
                'url': url,
                'fullpath': each
            }
        return images_dict

    @staticmethod
    def _get_annotations():
        xml_dict = {}
        xml_files = GetNewImage._parser_folder('static/data/annotations/', ['.xml'])
        for each in xml_files:
            url = each.split(PWD)[-1].lstrip('/')
            file = os.path.basename(url)
            # print(url, file)
            xml_dict[file] = {
                'url': url,
                'fullpath': each
            }
        return xml_dict


In [2]:
PWD

'/Users/sampathm/devbox/vitaFlow/vitaflow/annotate_server'

In [3]:
GetNewImage.refresh()

In [4]:
GetNewImage.XmlFiles.keys()

dict_keys(['street-animals-birds-doves.xml', 'pexels-photo-56733.xml', 'pexels-photo-416117.xml', 'gulls-seagull-wings-feather-56593.xml', 'bird-birds-ibis-flight-55817.xml', 'pexels-photo-66629.xml'])

In [5]:
GetNewImage.ImageFiles.keys()

dict_keys(['street-animals-birds-doves.jpg', 'pexels-photo-56733.jpg', 'pexels-photo-60091.jpg', 'pexels-photo-416117.jpg', 'pexels-photo-145380.jpg', 'pexels-photo-110812.jpg', 'pexels-photo-151048.jpg', 'gulls-seagull-wings-feather-56593.jpg', 'bird-birds-ibis-flight-55817.jpg', 'pexels-photo-66629.jpg'])

In [6]:
GetNewImage.PendingImages

['pexels-photo-110812',
 'pexels-photo-151048',
 'pexels-photo-145380',
 'pexels-photo-60091']

In [7]:
GetNewImage.CompletedImages

['pexels-photo-416117',
 'pexels-photo-66629',
 'gulls-seagull-wings-feather-56593',
 'bird-birds-ibis-flight-55817',
 'pexels-photo-56733',
 'street-animals-birds-doves']

In [8]:
GetNewImage.get_new_image()

{'id': 'pexels-photo-145380.jpg',
 'url': 'static/data/images/pexels-photo-145380.jpg',
 'folder': '',
 'annotations': []}

In [9]:
GetNewImage.refresh()
GetNewImage.PendingImages

['pexels-photo-110812',
 'pexels-photo-151048',
 'pexels-photo-145380',
 'pexels-photo-60091']

In [10]:
!ls -l static/data/annotations/

total 40
-rw-r--r--  1 sampathm  staff  955 Jan 31 11:07 bird-birds-ibis-flight-55817.xml
-rw-r--r--  1 sampathm  staff  642 Jan 31 11:07 gulls-seagull-wings-feather-56593.xml
-rw-r--r--  1 sampathm  staff  632 Jan 31 11:07 pexels-photo-416117.xml
-rw-r--r--  1 sampathm  staff  627 Jan 31 11:49 pexels-photo-56733.xml
-rw-r--r--  1 sampathm  staff  636 Jan 31 11:06 pexels-photo-66629.xml
-rw-r--r--  1 sampathm  staff    0 Feb  7 18:47 street-animals-birds-doves.xml
-rw-r--r--  1 sampathm  staff    0 Feb  1 17:00 temp


In [11]:
!touch static/data/annotations/street-animals-birds-doves.xml

In [12]:
GetNewImage.refresh()
GetNewImage.PendingImages

['pexels-photo-110812',
 'pexels-photo-151048',
 'pexels-photo-145380',
 'pexels-photo-60091']

In [1]:
import annotate

In [8]:
!ls -l static/data/annotations/

total 48
-rw-r--r--  1 sampathm  staff  955 Jan 31 11:07 bird-birds-ibis-flight-55817.xml
-rw-r--r--  1 sampathm  staff  642 Jan 31 11:07 gulls-seagull-wings-feather-56593.xml
-rw-r--r--  1 sampathm  staff  615 Feb  7 19:00 pexels-photo-110812.xml
-rw-r--r--  1 sampathm  staff  632 Jan 31 11:07 pexels-photo-416117.xml
-rw-r--r--  1 sampathm  staff  627 Jan 31 11:49 pexels-photo-56733.xml
-rw-r--r--  1 sampathm  staff  636 Jan 31 11:06 pexels-photo-66629.xml


In [9]:
annotate.validate_tags_and_regions({'sendInfo': '{"url":"static/data/images/pexels-photo-110812.jpg","id":"pexels-photo-110812.jpg","folder":"","width":1135,"height":750,"annotations":[{"tag":"Eagle","x":419.547123623011,"y":168.05555555555554,"width":298.6842105263158,"height":416.66666666666663}]}'})

<?xml version="1.0" ?>
<annotation>
	<folder/>
	<filename>pexels-photo-110812.jpg</filename>
	<path>static/data/images/pexels-photo-110812.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size_part>
		<width>1135</width>
		<height>750</height>
		<depth>3</depth>
	</size_part>
	<segmented>0</segmented>
	<object>
		<name>Eagle</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>419.547123623011</xmin>
			<ymin>168.05555555555554</ymin>
			<xmax>718.2313341493268</xmax>
			<ymax>584.7222222222222</ymax>
		</bndbox>
	</object>
</annotation>



In [10]:
!ls -l static/data/annotations/

total 48
-rw-r--r--  1 sampathm  staff  955 Jan 31 11:07 bird-birds-ibis-flight-55817.xml
-rw-r--r--  1 sampathm  staff  642 Jan 31 11:07 gulls-seagull-wings-feather-56593.xml
-rw-r--r--  1 sampathm  staff  615 Feb  7 19:01 pexels-photo-110812.xml
-rw-r--r--  1 sampathm  staff  632 Jan 31 11:07 pexels-photo-416117.xml
-rw-r--r--  1 sampathm  staff  627 Jan 31 11:49 pexels-photo-56733.xml
-rw-r--r--  1 sampathm  staff  636 Jan 31 11:06 pexels-photo-66629.xml
